In [7]:
import pandas as pd
import tabula as tb
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import re

pd.options.display.max_rows = None
pd.options.display.max_columns = None

df_page=pd.DataFrame([])
df_combine=pd.DataFrame([])

file = '/Users/talib/Documents/Projects/OrbitalWitness/Data/Official_Copy_Register_EGL363613.pdf'

#count number of pages in pdf
def count_pdf_pages(file):
   rxcountpages = re.compile(r"/Type\s*/Page([^s]|$)", re.MULTILINE|re.DOTALL)
   with open(file, "rb") as temp_file:
     return len(rxcountpages.findall(temp_file.read().decode('latin-1')))
   
# get number of pages in variable
pages = int(count_pdf_pages(file)) 

# loop over number of pages
for pageiter in range(pages):
#    define area and columns for tabular information
   df_page= tb.read_pdf(file, pages = pageiter+1, area=[120,0,820,900],columns = [80,180, 350, 450, 520], pandas_options={'header': None}, stream=True)
#    additional check for valid data in pdf
   if len(df_page) > 0: 
     df_combine = df_combine._append(df_page)

# necessary conversion before putting it in a dataframe
datas= np.array(df_combine, dtype='object')
# convert to dataframe 
df = pd.DataFrame(datas)


#discard all data before and including header
df= df.iloc[(df.loc[df[1]=='and plan ref.'].index[0]+1):, :]

# define first occurrence of record based on entry number/record number
df['border'] = df.apply(lambda x: 1 if re.findall('^[+-]?(\d*\.)?\d+$', str(x[0])) else 0, axis = 1)
# repeat row number for each record
df['row'] = df['border'].transform('cumsum')
# clean nans
df = df.replace(np.NaN,'')
# mark the row with note 
df['NoteRow'] = np.where(
   df[1].str.contains('NOTE'),  # Condition where there is note in record
   df['row'],  # Where True
   0 # Where False
)

# somehow bloody nans appeared again - clean it!
df = df.replace(np.NaN,'')

# Function to remove rows based on the condition
def remove_rows_after_pattern(df, pattern_column, pattern):
    # Boolean array to mark rows to keep
    keep_mask = [True] * len(df)
    
    # Dictionary to keep track of the first occurrence of pattern for each row number
    first_occurrence = {}
    
    for idx, (row_num, value) in enumerate(zip(df['row'], df[pattern_column])):
        if row_num in first_occurrence:
            # If we've already seen this row_num and the first occurrence of pattern, mark to drop
            keep_mask[idx] = False
        elif 'NOTE' in value:
            # Mark the first occurrence of the pattern
            first_occurrence[row_num] = idx
            keep_mask[idx] = False
    
    # Apply the mask to the DataFrame
    df_filtered = df[keep_mask]
    
    return df_filtered

# Remove all rows that contain note information from main dataframe
result_df = remove_rows_after_pattern(df, 1, 'NOTE')


# concatenate notes into one row
dfnew = pd.merge(left=df[[0,1,2,3,4,'row','NoteRow']], right=df['NoteRow'], how='left', left_on='row', right_on='NoteRow')
# bloody nans
dfnew = dfnew.replace(np.NaN,'')
# will remove the rows that don't have notes information in it
df_noterows = dfnew[dfnew['NoteRow_y'] != '']

# Group by 'Category' and concatenate all columns into a single string
# Define the custom aggregation function for specific columns
def concat_selected_columns(group, columns):
    concatenated_string = ' '.join(group[columns].astype(str).values.flatten())
    match = re.search(r'\bNOTE\b', concatenated_string)
    if match:
        return concatenated_string[match.start():].strip()
    return ''


# List of columns to concatenate
columns_to_concatenate = [1,2,3,4]

# Group by 'note row number' and concatenate specified columns into a single string - this dataframe only has noterows 
result = df_noterows.groupby('NoteRow_y').apply(lambda group: concat_selected_columns(group, columns_to_concatenate)).reset_index().astype(str)

# Rename the columns for clarity
result.columns = ['NoteRow', 'ConcatenatedValues']
# clean the dataframe to remove leading and trailing spaces
result_df = result_df.replace(r"^ +| +$", r"", regex=True)
# clean the record number from string/float to integer
result_df[0] = pd.to_numeric(result_df[0], errors='coerce').astype('Int64')
# clean the note row number from float to integer
result['NoteRow'] = pd.to_numeric(result['NoteRow'], errors='coerce').astype('Int64')

# merge the original dataframe with notes dataframe - join on note number/record number
new_df = pd.merge(left=result_df[[0,1,2,3,4,'row','NoteRow']], right=result[['NoteRow','ConcatenatedValues']], how='left', left_on=0, right_on='NoteRow')

schedule_formatted = pd.DataFrame()
schedule = new_df[[0,1, 2, 3,4,'ConcatenatedValues', 'row']]
schedule = schedule[schedule[1].notnull()]
# perform group by to group for index
schedule['index'] = schedule.groupby('row').cumcount()+1
schedule = schedule[schedule['row']!=0]
schedule['ConcatenatedValues'] = schedule['ConcatenatedValues'].replace(np.NaN,'')
# now pivot table based on row number - vertical to horizontal 
schedule = schedule.pivot_table(index = ['row'], columns = ['index'], values = [1,2,3,4,'ConcatenatedValues'],aggfunc=lambda x: ' '.join(x)).reset_index()
schedule = schedule.replace(np.NaN,'')
schedule_formatted['Registration date and plan ref.'] = schedule.iloc[0:,1:8].apply(lambda x: ''.join(map(str, x)), axis=1)
schedule_formatted['Property description'] = schedule.iloc[0:,9:16].apply(lambda x: ''.join(map(str, x)), axis=1)
schedule_formatted['Date of lease and term.'] = schedule.iloc[0:,17:24].apply(lambda x: ''.join(map(str, x)), axis=1)
schedule_formatted['Lessee''s title.'] = schedule.iloc[0:,25:32].apply(lambda x: ''.join(map(str, x)), axis=1)
schedule_formatted['Notes.'] = schedule.iloc[0:,33:40].apply(lambda x: ''.join(map(str, x)), axis=1)
display(schedule_formatted)

/var/folders/h3/cdnw_1_n2bl3zdcfsknxp4jw0000gn/T/ipykernel_16580/887491540.py:105: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df_noterows.groupby('NoteRow_y').apply(lambda group: concat_selected_columns(group, columns_to_concatenate)).reset_index().astype(str)


,Registration date and plan ref.,Property description,Date of lease and term.,Lessees title.,Notes.
0,28.01.2009tinted blue(part of),Transformer Chamber (GroundFloor),23.01.200999 years from23.1.2009,EGL551039,
1,09.07.2009Edged andnumbered 2 inblue (part of),"Endeavour House, 47 CubaStreet, London",06.07.2009125 years from1.1.2009,EGL557357,
2,25.09.2009Edged andnumbered 3 inblue (part of),"Mayflower House, WestferryRoad, London",10.09.2009125 years from1.1.2009,EGL560877,
3,16.12.2009Edged andnumbered 4 inblue (part of),"Flat 1602, Landmark WestTower(sixteenth floor)",12.11.2009999 years from1.1.2009,EGL565026,
4,17.12.2009Edged andnumbered 4 inblue (part of),"Flat 207, Landmark WestTower (second floor flat)",01.12.2009999 years from1.1.2009,EGL565086,
5,17.12.2009Edged andnumbered 4 inblue (part of),"Flat 704, Landmark WestTower (seventh floor flat)",04.12.2009999 years from1.1.2009,EGL565089,
6,18.12.2009Edged andnumbered 4 inblue (part of),Flat 2207 Landmark WestTower (twenty second fl...,01.12.2009999 years from1.1.2009,EGL565182,
7,23.12.2009Edged andnumbered 4 inblue (part of),Flat 1201 Landmark WestTower (twelfth floor flat),01.12.2009999 years from1.1.2009,EGL565292,
8,23.12.2009Edged andnumbered 4 inblue (part of),Flat 403 Landmark WestTower (fourth floor flat),27.11.2009999 years from1.1.2009,EGL565293,
9,05.01.2010Edged andnumbered 4 inblue (part of),Flat 1105 Landmark WestTower (eleventh floor f...,01.12.2009999 years from1.1.2009,EGL565470,
